# finrl_train (SAC)
Переработанная версия тренировки под SAC.

In [2]:
!pip install tensorboard

  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.5 MB 5.8 MB/s eta 0:00:01
   -------------------- ------------------- 2.9/5.5 MB 7.5 MB/s eta 0:00:01
   ------------------------------ --------- 4.2/5.5 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 5.5/5.5 MB 8.2 MB/s eta 0:00:00
Using cached tensorboard_data_server-0.7.2-py3-none-any.whl (2.4 kB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   -------------------- ------------------- 2.4/4.7 MB 12.0 MB/s eta 0:00:01
   ----------------------------------- ---- 4.2/4.7 MB 10.0 MB/s eta 0:00:01


In [1]:
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import os
from pathlib import Path
import torch

# ✅ ПРОВЕРКА CUDA
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ✅ ПАПКИ
TRAINED_MODELS_DIR = Path("trained_models")
RESULTS_DIR = Path("results/sac_multi_stock")
TRAINED_MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# 1) Загрузка CSV
def load_stock_data(folder='prepared_csv'):
    all_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.csv'):
                all_files.append(os.path.join(root, file))
    
    print(f"Found {len(all_files)} ")
    if not all_files:
        return None
    
    dfs = []
    for file_path in all_files:
        ticker = os.path.basename(file_path).replace('.csv', '')
        print(f" {ticker}")
        
        df = pd.read_csv(file_path)
        date_col = next((c for c in ['date', 'Date'] if c in df.columns), None)
        close_col = next((c for c in ['close', 'Close', 'adj close'] if c in df.columns), None)
        
        if date_col and close_col:
            df = df[[date_col, close_col]].copy()
            df['date'] = pd.to_datetime(df[date_col])
            df['tic'] = ticker
            df['close'] = df[close_col].astype(float)
            df = df[['date', 'tic', 'close']].dropna()
            dfs.append(df)
    
    df_all = pd.concat(dfs, ignore_index=True).sort_values(['date', 'tic']).reset_index(drop=True)
    print(f" {df_all['tic'].nunique()} stocks, {len(df_all):,} strings")
    return df_all

# 2) SAC Stock Env
class StockTradingEnv(gym.Env):
    def __init__(self, df):
        super().__init__()
        self.df = df.sort_values(['date', 'tic']).reset_index(drop=True)
        self.tickers = sorted(self.df['tic'].unique())
        self.stock_dim = len(self.tickers)
        self.dates = sorted(self.df['date'].unique())
        self.max_days = len(self.dates)
        
        self.current_day = 0
        self.initial_cash = 1000000
        self.cash = self.initial_cash
        self.holdings = np.zeros(self.stock_dim)
        
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(1 + self.stock_dim,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(self.stock_dim,), dtype=np.float32)
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_day = 0
        self.cash = self.initial_cash
        self.holdings = np.zeros(self.stock_dim)
        return self._get_obs(), {}
    
    def _get_obs(self):
        day_data = self.df[self.df['date'] == self.dates[self.current_day]]
        prices = day_data.set_index('tic')['close'].reindex(self.tickers).fillna(method='ffill').values
        prices = prices / (prices.mean() + 1e-6)
        return np.concatenate([[self.cash / self.initial_cash], prices]).astype(np.float32)
    
    def step(self, action):
        self.current_day += 1
        terminated = self.current_day >= self.max_days - 1
        truncated = False
        
        day_data = self.df[self.df['date'] == self.dates[self.current_day]]
        prices = day_data.set_index('tic')['close'].reindex(self.tickers).fillna(method='ffill').values
        
        portfolio_value = self.cash + np.sum(self.holdings * prices)
        for i in range(self.stock_dim):
            target_weight = np.clip((action[i] + 1) / 2, 0, 1)
            target_value = portfolio_value * target_weight * 0.1
            target_shares = target_value / (prices[i] + 1e-6)
            self.holdings[i] = target_shares * 0.1
        
        new_portfolio_value = self.cash + np.sum(self.holdings * prices)
        reward = np.log(new_portfolio_value / self.initial_cash)
        
        return self._get_obs(), reward, terminated, truncated, {'portfolio_value': new_portfolio_value}

# 3) 🚀 SAC ОБУЧЕНИЕ НА GPU
print(" SAC Stock Trading (GPU)")
df = load_stock_data('prepared_csv')
if df is not None:
    env_fn = lambda: StockTradingEnv(df)
    env_train = DummyVecEnv([env_fn])
    
    # ✅ CUDA GPU!
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")
    
    model = SAC(
        "MlpPolicy", 
        env_train,
        verbose=1,
        learning_rate=3e-4,
        buffer_size=100000,
        batch_size=256,
        device=device  # ✅ GPU ускорение!
    )
    
    model.learn(total_timesteps=100000, progress_bar=True)
    
    model_path = TRAINED_MODELS_DIR / "sac_stocks.zip"
    model.save(model_path)
    print(f"SAC : {model_path}")
    
    print(f"Results: {RESULTS_DIR}")



Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


CUDA Available: False
 SAC Stock Trading (GPU)
Found 12 
 AAPL_prepared
 AMZN_prepared
 BAC_prepared
 DIS_prepared
 GOOG_prepared
 JPM_prepared
 META_prepared
 MSFT_prepared
 NFLX_prepared
 NVDA_prepared
 TSLA_prepared
 WMT_prepared
 12 stocks, 2,400 strings
Device: cpu
Using cpu device


Output()

C:\Users\cody\AppData\Local\Temp\ipykernel_25688\1182738268.py:92: FutureWarning: Series.fillna with 'method' is 
deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  prices = day_data.set_index('tic')['close'].reindex(self.tickers).fillna(method='ffill').values

C:\Users\cody\AppData\Local\Temp\ipykernel_25688\1182738268.py:82: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  prices = day_data.set_index('tic')['close'].reindex(self.tickers).fillna(method='ffill').values


...

---------------------------------
| time/              |          |
|    episodes        | 488      |
|    fps             | 32       |
|    time_elapsed    | 2986     |
|    total_timesteps | 97112    |
| train/             |          |
|    actor_loss      | -4.63    |
|    critic_loss     | 0.0367   |
|    ent_coef        | 0.000592 |
|    ent_coef_loss   | 1.13     |
|    learning_rate   | 0.0003   |
|    n_updates       | 97011    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 492      |
|    fps             | 32       |
|    time_elapsed    | 3016     |
|    total_timesteps | 97908    |
| train/             |          |
|    actor_loss      | -4.61    |
|    critic_loss     | 0.0552   |
|    ent_coef        | 0.000563 |
|    ent_coef_loss   | -15.8    |
|    learning_rate   | 0.0003   |
|    n_updates       | 97807    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 496      |
|    fps             | 32       |
|    time_elapsed    | 3046     |
|    total_timesteps | 98704    |
| train/             |          |
|    actor_loss      | -4.73    |
|    critic_loss     | 0.0409   |
|    ent_coef        | 0.000753 |
|    ent_coef_loss   | 43.2     |
|    learning_rate   | 0.0003   |
|    n_updates       | 98603    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 500      |
|    fps             | 32       |
|    time_elapsed    | 3076     |
|    total_timesteps | 99500    |
| train/             |          |
|    actor_loss      | -4.53    |
|    critic_loss     | 0.0345   |
|    ent_coef        | 0.000945 |
|    ent_coef_loss   | 17       |
|    learning_rate   | 0.0003   |
|    n_updates       | 99399    |
---------------------------------


SAC : trained_models\sac_stocks.zip
Results: results\sac_multi_stock
